## Data Scraping 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
"""
Opening a Google Chrome browser simulation, to get complete web page's code including dynamically rendered content.  
"""
browser= webdriver.Chrome('./chromedriver.exe')

In [3]:
browser.delete_all_cookies()

In [4]:
browser.get('https://www.imdb.com/chart/top?ref_=nv_mv_250') ## Navigating to the required page.

In [5]:
## Initializing BeautifulSoup object to extract information from HTML File.
soup= BeautifulSoup(browser.page_source, "html.parser") 

In [6]:
"""
Finding all the <table> tags, where class="chart full-width", this table contains all the data of movie. 
This was determined by observing the source code of the given webpage. 
Using find_all instead of find to be double sure, if any there are any other tables with the same characteristics. 

"""

movie_table=soup.find_all('table',class_="chart full-width")

In [7]:
len(movie_table)

1

In [8]:
movie_data=movie_table[0].find_all('tbody')[0].find_all('tr')

In [9]:
len(movie_data)

250

In [10]:
## Defining the list that will have all appropiate data after processing. (Format: [Name, Year, Rating, Thumbnail URL])
required_data=[]

In [11]:
## Below all the unique characteristics of the required information are derived by observation.
for each in movie_data:
    required_data.append([
        each.find('td',class_="titleColumn").find("a").text, ## Name
        int(each.find('span',class_="secondaryInfo").text[1:-1]), ## Year
        float(each.find('td',class_="ratingColumn imdbRating").find('strong').text), ## Rating
        each.find('td',class_="posterColumn").find('img').get('src') ## Thumbnail URL
    ])

In [12]:
required_data

[['The Shawshank Redemption',
  1994,
  9.2,
  'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UY67_CR0,0,45,67_AL_.jpg'],
 ['The Godfather',
  1972,
  9.1,
  'https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY67_CR1,0,45,67_AL_.jpg'],
 ['The Godfather: Part II',
  1974,
  9.0,
  'https://m.media-amazon.com/images/M/MV5BMWMwMGQzZTItY2JlNC00OWZiLWIyMDctNDk2ZDQ2YjRjMWQ0XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY67_CR1,0,45,67_AL_.jpg'],
 ['The Dark Knight',
  2008,
  9.0,
  'https://m.media-amazon.com/images/M/MV5BMTMxNTMwODM0NF5BMl5BanBnXkFtZTcwODAyMTk2Mw@@._V1_UY67_CR0,0,45,67_AL_.jpg'],
 ['12 Angry Men',
  1957,
  8.9,
  'https://m.media-amazon.com/images/M/MV5BMWU4N2FjNzYtNTVkNC00NzQ0LTg0MjAtYTJlMjFhNGUxZDFmXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX45_CR0,0,45,67_AL_.jpg'],
 ["Schindler's List",
  1993,
  8.9,
  'https://m.media-amazon.com/images/M/MV5BN

In [ ]:
import pandas as pd

In [ ]:
## Saving the data in csv for further usage. 
pd.DataFrame(required_data,columns=['Name','Year','Rating','Thumbnail URL']).to_csv('./imdb-movies-data.csv', index=False)

## Creating Database

In [ ]:
import sqlite3

In [ ]:
connection= sqlite3.connect('imdb.db') ## Connecting to './imdb.db', new database will be created if it does not exsist.

In [ ]:
cur= connection.cursor()

In [ ]:
## This line is only written for convenience in case, if the code is again run.
cur.execute("DROP TABLE IF EXISTS 'movies'")

In [ ]:
"""
The code below clearly depicts the schema of the table.

thumbnail_url is selected as the primary key because two movies can have same Name, Year and Rating. 
For example:
[1] Chaos(2005): https://www.imdb.com/title/tt0402910/
[2] Chaos(2005): https://www.imdb.com/title/tt0405977/

The above two movies have same Name and Year of release. Therefore, thumbnail i.e. poster of the movie will be unqiue. 

"""
create_movies_table = """ CREATE TABLE movies (
                                        name text NOT NULL,
                                        year integer NOT NULL,
                                        rating real NOT NULL,
                                        thumbnail_url text PRIMARY KEY
                                    ); """

In [ ]:
cur.execute(create_movies_table)

In [ ]:
cur.executemany("INSERT INTO movies VALUES (?,?,?,?);",required_data) ## Inserting the complete data into the table

In [ ]:
## Confirming if the data is properly inserted and number of rows are equal to the original movies in the data. 
cur.execute("SELECT * FROM movies;")
len(cur.fetchall())

In [ ]:
connection.commit() ## Commiting all the changes made in the database, if it is not run the changes will not be saved. 

In [ ]:
connection.close()